# Drzewa decyzyjne

In [ ]:
import csv

with open('titanic-homework.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        print(row) # wywalic nagłówek


## Komentarz




- Aleksander Kamiński 155840
- Piotr Nowacki 15